In [ ]:
!pip install ultralytics 

In [ ]:
import os
import re
import csv
from datetime import datetime

import torch
from ultralytics import YOLO
from PIL import Image

Config

In [ ]:
IMAGE_ROOT = "images"   # folder chứa cam1/, cam2/, ...
OUTPUT_CSV = "yolo_counts.csv"

MOTORBIKE_MODEL = "yolov8s_motorbike_best.pt"  # fine-tuned
VEHICLE_MODEL = "yolov8s.pt"                   # pretrained COCO

CONF_MOTORBIKE = 0.3
CONF_VEHICLE = 0.4
IMG_SIZE = 480        # CPU-friendly
DEVICE = "cpu"        # ép CPU cho chắc

# COCO class ids
COCO_CAR = 2
COCO_MOTORBIKE = 3
COCO_BUS = 5
COCO_TRUCK = 7

Utils

In [ ]:
def parse_timestamp(filename):
    """
    Expect filename like:
    2025-01-12_06-45-10.jpg
    """
    match = re.search(r"\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2}", filename)
    if not match:
        raise ValueError(f"Cannot parse timestamp from {filename}")
    return datetime.strptime(match.group(), "%Y-%m-%d_%H-%M-%S")


def count_motorbike(model, image_path):
    results = model.predict(
        source=image_path,
        conf=CONF_MOTORBIKE,
        imgsz=IMG_SIZE,
        device=DEVICE,
        verbose=False
    )
    return len(results[0].boxes)


def count_other_vehicles(model, image_path):
    results = model.predict(
        source=image_path,
        conf=CONF_VEHICLE,
        imgsz=IMG_SIZE,
        device=DEVICE,
        verbose=False
    )

    car = bus = truck = 0

    for cls in results[0].boxes.cls.tolist():
        cls = int(cls)
        if cls == COCO_CAR:
            car += 1
        elif cls == COCO_BUS:
            bus += 1
        elif cls == COCO_TRUCK:
            truck += 1

    return car, bus, truck


Main

In [ ]:
def main():
    print("🚀 Loading YOLO models...")
    motorbike_model = YOLO(MOTORBIKE_MODEL)
    vehicle_model = YOLO(VEHICLE_MODEL)

    rows = []

    for cam in sorted(os.listdir(IMAGE_ROOT)):
        cam_path = os.path.join(IMAGE_ROOT, cam)
        if not os.path.isdir(cam_path):
            continue

        print(f"\n📷 Processing {cam}...")

        for img_name in sorted(os.listdir(cam_path)):
            if not img_name.lower().endswith((".jpg", ".png")):
                continue

            img_path = os.path.join(cam_path, img_name)
            timestamp = parse_timestamp(img_name)

            motorbike = count_motorbike(motorbike_model, img_path)
            car, bus, truck = count_other_vehicles(vehicle_model, img_path)

            total = motorbike + car + bus + truck
            rows.append([
                cam,
                timestamp.strftime("%Y-%m-%d %H:%M:%S"),
                motorbike,
                car,
                bus,
                truck, 
                total
            ])

    # write CSV
    print("\n💾 Writing CSV...")
    with open(OUTPUT_CSV, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["camera_id", "timestamp", "motorbike", "car", "bus", "truck", "total"])
        writer.writerows(rows)

    print(f"✅ DONE. Saved to {OUTPUT_CSV}")
    print(f"Total frames processed: {len(rows)}")


if __name__ == "__main__":
    main()